This ipynb performs the last dea of the pipeline
- comparing each mono-culture sample to the co-culture
- comparing each cluster of the mono-co-culture combination against each other
- plots a volcano plot of the sample-to-sample dea

In [ ]:
import scanpy as sc
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import os
from bioinfokit import analys, visuz

In [ ]:
# astroco
path_to_adata = 'C:/Users/julia/Project/Output_directory4/integrated_BL_C_BL_A/anndata_storage/BL_C_BL_A_subset_anndata.h5ad'
astroco = sc.read_h5ad(os.path.join(path_to_adata))
# neuroco
path_to_adata = 'C:/Users/julia/Project/Output_directory4/integrated_BL_C_BL_N/anndata_storage/BL_C_BL_N_subset_anndata.h5ad'
neuroco = sc.read_h5ad(os.path.join(path_to_adata))

In [ ]:
astroco = astroco.raw.to_adata()
neuroco = neuroco.raw.to_adata()

Perform DEA with mono-culture against co-culture for both samples

In [ ]:
sc.tl.rank_genes_groups(astroco, 
                        groupby='batch', 
                        groups=['BL_C'],
                        method='wilcoxon',
                        corr_method='bonferroni',
                        pts=True)
sc.tl.rank_genes_groups(neuroco, 
                        groupby='batch', 
                        groups=['BL_C'],
                        method='wilcoxon',
                        corr_method='bonferroni',
                        pts=True)

In [ ]:
sc.pl.rank_genes_groups(astroco, sharey=False, show=False, title='BL_C vs BL_A')
sc.pl.rank_genes_groups(neuroco, sharey=False, show=False, title='BL_C vs BL_N')

In [ ]:
# store results in tsv, for now take ALL p-value scores just like Maurits
BLCAdf = sc.get.rank_genes_groups_df(astroco, group='BL_C', key='rank_genes_groups', pval_cutoff=None, log2fc_min=None, log2fc_max=None, gene_symbols=None)
BLCNdf = sc.get.rank_genes_groups_df(neuroco, group='BL_C', key='rank_genes_groups', pval_cutoff=None, log2fc_min=None, log2fc_max=None, gene_symbols=None)
BLCAdf.to_csv(os.path.join('C:/Users/julia/Project/big_dea_output/', 'BLCA_rank_genes_groups.tsv'), sep='\t', encoding='utf-8')  
BLCNdf.to_csv(os.path.join('C:/Users/julia/Project/big_dea_output/', 'BLCN_rank_genes_groups.tsv'), sep='\t', encoding='utf-8')  

Perform DEA in of the samples per cluster

In [ ]:
sc.tl.rank_genes_groups(astroco, 'leiden', method='wilcoxon', corr_method='bonferroni', key='wilcoxon', pts=True, key_added='rank_genes_groups_leiden')
sc.tl.rank_genes_groups(neuroco, 'leiden', method='wilcoxon', corr_method='bonferroni', key='wilcoxon', pts=True, key_added='rank_genes_groups_leiden')

In [ ]:
sc.pl.rank_genes_groups(astroco, sharey=False, show=False, key='rank_genes_groups_leiden')
sc.pl.rank_genes_groups(neuroco, sharey=False, show=False, key='rank_genes_groups_leiden')


In [ ]:
# store results in tsv, for now take ALL p-value scores just like Maurits
cluster = range(len(astroco.obs['leiden'].unique()))
cluster = [str(x) for x in cluster]
BLCAdf = sc.get.rank_genes_groups_df(astroco, group=cluster, key='rank_genes_groups_leiden', pval_cutoff=None, log2fc_min=None, log2fc_max=None, gene_symbols=None)
cluster = range(len(neuroco.obs['leiden'].unique()))
cluster = [str(x) for x in cluster]
BLCNdf = sc.get.rank_genes_groups_df(neuroco, group=cluster, key='rank_genes_groups_leiden', pval_cutoff=None, log2fc_min=None, log2fc_max=None, gene_symbols=None)
BLCAdf.to_csv(os.path.join('C:/Users/julia/Project/big_dea_output/', 'BLCA_leiden_rank_genes_groups.tsv'), sep='\t', encoding='utf-8')  
BLCNdf.to_csv(os.path.join('C:/Users/julia/Project/big_dea_output/', 'BLCN_leiden_rank_genes_groups.tsv'), sep='\t', encoding='utf-8')  

Volcano plot for both samples:

In [ ]:
# I re-ran DEA for mono-culture against co-culture just to be sure
visuz.GeneExpression.volcano(df=BLCAdf, lfc="logfoldchanges", pv="pvals_adj", plotlegend=True, legendpos='upper right',
    color=('red', 'black', 'blue'), geneid="names", legendanchor=(1.46,1),figname='Volcano plot Astroco',
    gstyle=2, sign_line=True, xlm=(-50,50,5), ylm=(-50, 310,50), figtype='svg', axtickfontsize=10,
    lfc_thr=(1, 1), pv_thr=(0.05, 0.05), show=True)
visuz.GeneExpression.volcano(df=BLCNdf, lfc="logfoldchanges", pv="pvals_adj", plotlegend=True, legendpos='upper right',
    color=('red', 'black', 'blue'), geneid="names", legendanchor=(1.46,1), figname='Volcano plot Neuroco',
    gstyle=2, sign_line=True, xlm=(-50,50,5), ylm=(-50, 310,50), figtype='svg', axtickfontsize=10,
    lfc_thr=(1, 1), pv_thr=(0.05, 0.05), show=True)